### Test Corr Models

In [4]:
import numpy as np
import pandas as pd
from copy import deepcopy
import yfinance as yf

import dynamic_correlations as CM
import expected_shortfall as ES
import backtesting as BT
import plots as IP
import data_download as dd

### Data 

In [ ]:
'''# Step 1: Download prices
tickers = ["^GSPC", "^STOXX50E", "^N225"]
data = yf.download(tickers, start="2015-01-01", end="2025-01-01")["Close"]
data = data.dropna()

# Step 2: Define fixed quantities
quantities = pd.Series([10, 80, -10], index=tickers)

# Step 3: Compute monetary positions
x_matrix = data.multiply(quantities, axis=1)'''

[*********************100%***********************]  3 of 3 completed


In [5]:
# Define tickers and download price data
tickers = ["MSFT", "NVDA", "AAPL"]
prices = dd.get_raw_prices(tickers, start="2024-01-01")


# Define portfolio — 3 shares of each asset
shares = pd.Series({"MSFT": 3, "NVDA": 3, "AAPL": 3})

# Compute daily value of each position
x_matrix = prices * shares

In [6]:
x_matrix.head()

,AAPL,MSFT,NVDA
Date,,,
2024-01-02,553.596222,1102.141846,144.449753
2024-01-03,549.451126,1101.339478,142.653431
2024-01-04,542.473114,1093.434631,143.939953
2024-01-05,540.296036,1092.869843,147.235691
2024-01-08,553.357681,1113.494019,156.700138


In [7]:
# Run MA Correlation VaR
x_ma = deepcopy(x_matrix)
result_ma= CM.var_corr_moving_average(x_ma, window_size=20)

# Run RiskMetrics Correlation VaR
x_rm = deepcopy(x_matrix)
result_rm = CM.var_corr_ewma(x_rm)


In [9]:
result_ma.head()

,Returns,Volatility,VaR,VaR Monetary,VaR Violation
2024-01-31,-0.024086,0.010765,0.025044,47.981949,False
2024-02-01,0.015802,0.010977,0.025536,49.698089,False
2024-02-02,0.014881,0.010880,0.025310,49.979922,False
2024-02-05,-0.000430,0.010946,0.025465,50.244220,False
2024-02-06,0.000570,0.009940,0.023124,45.649643,False


In [10]:
result_rm.head()

,Returns,Volatility,VaR,VaR Monetary,VaR Violation
2024-01-03,-0.003730,0.014881,0.034619,62.087315,False
2024-01-04,-0.007551,0.014558,0.033867,60.277943,False
2024-01-05,0.000359,0.014130,0.032872,58.525659,False
2024-01-08,0.024383,0.014972,0.034830,63.514979,False
2024-01-09,0.002589,0.014543,0.033833,61.854612,False


In [11]:
# Apply volatility-based ES to the MA VaR result
result_ma = ES.es_correlation(result_ma, confidence_level=0.99)

# Same for RiskMetrics
result_rm = ES.es_correlation(result_rm, confidence_level=0.99)

In [ ]:
# Plot MA model results
fig_ma_es = IP.plot_backtest(result_ma, subset=("2019-11-30", "2020-11-30"), interactive=False)

# Plot RiskMetrics model results
fig_rm_es = IP.plot_backtest(result_rm, interactive=False)


In [12]:
violations_ma, rate_ma = BT.count_violations(result_ma)

print("== MA Correlation VaR Backtest ==")
print(f"Total Violations: {violations_ma}")
print(f"Violation Rate: {rate_ma:.2f}%")

== MA Correlation VaR Backtest ==
Total Violations: 8
Violation Rate: 2.50%


In [13]:
violations_rm, rate_rm = BT.count_violations(result_rm)

print("== RiskMetrics Correlation VaR Backtest ==")
print(f"Total Violations: {violations_rm}")
print(f"Violation Rate: {rate_rm:.2f}%")

== RiskMetrics Correlation VaR Backtest ==
Total Violations: 5
Violation Rate: 1.47%


In [19]:
fig_ma_vol = IP.plot_volatility(result_ma["Volatility"])
fig_rm_vol = IP.plot_volatility(result_rm["Volatility"])

### Bakctesting 

In [14]:
# 1. Recompute monetary positions (already done)
x_rm = deepcopy(x_matrix)

# 2. Run RiskMetrics Correlation VaR
result_rm = CM.var_corr_ewma(x_rm)

# 3. Count violations
total_violations, violation_rate = BT.count_violations(result_rm)

# 4. Run Kupiec test
confidence_level = 0.99  # adjust if needed
kupiec_results = BT.kupiec_test(total_violations, len(result_rm), confidence_level)

# 5. Run Christoffersen test
christoffersen_results = BT.christoffersen_test(result_rm)

# 6. Joint test
joint_results = BT.joint_lr_test(
    LR_uc=kupiec_results["LR_uc"],
    LR_c=christoffersen_results["LR_c"]
)


In [15]:
print("=== VaR Backtesting Summary ===")
print(f"Total Violations: {total_violations}")
print(f"Violation Rate: {violation_rate:.2f}%")

print("\n-- Kupiec Test --")
for k, v in kupiec_results.items():
    print(f"{k}: {v}")

print("\n-- Christoffersen Test --")
for k, v in christoffersen_results.items():
    print(f"{k}: {v}")

print("\n-- Joint Test --")
for k, v in joint_results.items():
    print(f"{k}: {v}")


=== VaR Backtesting Summary ===
Total Violations: 5
Violation Rate: 1.47%

-- Kupiec Test --
LR_uc: 0.6738158388514961
p_value: 0.41172438690362556
reject_null: False

-- Christoffersen Test --
LR_c: 0.15015579256778722
p_value: 0.6983865218030048
reject_null: False

-- Joint Test --
LR_total: 0.8239716314192833
p_value: 0.6623336707185933
reject_null: False


In [16]:
# 1. Recompute monetary positions (already done)
x_rm = deepcopy(x_matrix)

# 2. Run RiskMetrics Correlation VaR
result_rm = CM.var_corr_ewma(x_rm)

# 3. Count violations
total_violations, violation_rate = BT.count_violations(result_ma)

# 4. Run Kupiec test
confidence_level = 0.99  # adjust if needed
kupiec_results = BT.kupiec_test(total_violations, len(result_ma), confidence_level)

# 5. Run Christoffersen test
christoffersen_results = BT.christoffersen_test(result_ma)

# 6. Joint test
joint_results = BT.joint_lr_test(
    LR_uc=kupiec_results["LR_uc"],
    LR_c=christoffersen_results["LR_c"]
)


In [17]:
print("=== VaR Backtesting Summary ===")
print(f"Total Violations: {total_violations}")
print(f"Violation Rate: {violation_rate:.2f}%")

print("\n-- Kupiec Test --")
for k, v in kupiec_results.items():
    print(f"{k}: {v}")

print("\n-- Christoffersen Test --")
for k, v in christoffersen_results.items():
    print(f"{k}: {v}")

print("\n-- Joint Test --")
for k, v in joint_results.items():
    print(f"{k}: {v}")


=== VaR Backtesting Summary ===
Total Violations: 8
Violation Rate: 2.50%

-- Kupiec Test --
LR_uc: 5.133749100374487
p_value: 0.023464990449843848
reject_null: True

-- Christoffersen Test --
LR_c: 1.785738120540401
p_value: 0.18144629399279932
reject_null: False

-- Joint Test --
LR_total: 6.919487220914888
p_value: 0.03143782131378958
reject_null: True
